# ResNet

Fine tune an end-to-end ResNet model on the cars dataset.

In [226]:
# Whether this is a sample run or not
sample_run = False

In [227]:
# Packages
from PIL import Image
import os
import warnings
warnings.filterwarnings('ignore')
import torch
import torchvision.models as models
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import time

In [228]:
# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Set Up Images and Labels

Non-blurred

In [229]:
# Load data
directory_path = '../../../Data/Features/All Features/train/'
# list of files in directory
file_list = [f for f in os.listdir(directory_path) if f.endswith('.parquet')]
# read in all parquet files
combined_df = pd.concat([pd.read_parquet(directory_path + f, columns=['Class', 'harmonized_filename']) for f in file_list])
# replace .jpg with _no_blur.jpg
combined_df['harmonized_filename'] = combined_df['harmonized_filename'].str.replace('.jpg', '_no_blur.jpg')
# reset index
combined_df.reset_index(drop=True, inplace=True)
# create a dictionary mapping to encode the labels
label_mapping = {cl: num for num, cl in enumerate(combined_df['Class'].unique())}
print(label_mapping)
# save label mapping
pd.DataFrame(label_mapping.items(), columns=['Class', 'Encoded']).to_excel('../../../Output/Classifier Fitting/ResNet/label_mapping.xlsx', index=False)
# encode the labels
combined_df['Class'] = combined_df['Class'].map(label_mapping)
print(combined_df)

{'Sedan': 0, 'SUV': 1, 'Convertible': 2, 'Pickup': 3}
      Class                                harmonized_filename
0         0    Sedan_train_orig_test_01516_resized_no_blur.jpg
1         1     SUV_train_orig_train_00294_resized_no_blur.jpg
2         2  Convertible_train_orig_train_04236_resized_no_...
3         3  Pickup_train_orig_train_03906_resized_no_blur.jpg
4         1      SUV_train_orig_test_01344_resized_no_blur.jpg
...     ...                                                ...
5998      1      SUV_train_orig_test_06937_resized_no_blur.jpg
5999      0    Sedan_train_orig_test_02708_resized_no_blur.jpg
6000      0    Sedan_train_orig_test_05010_resized_no_blur.jpg
6001      0   Sedan_train_orig_train_02045_resized_no_blur.jpg
6002      2  Convertible_train_orig_test_02191_resized_no_b...

[6003 rows x 2 columns]


In [230]:
# Split into training and validation
# 80% training, 20% validation
# Use seed 290
train_df = combined_df.sample(frac=0.8, random_state=290)
val_df = combined_df.drop(train_df.index)
print(train_df)
print(val_df)

      Class                                harmonized_filename
2820      3      SUV_train_orig_test_03377_resized_no_blur.jpg
1940      1     SUV_train_orig_train_01474_resized_no_blur.jpg
5869      1      SUV_train_orig_test_00020_resized_no_blur.jpg
5294      0    Sedan_train_orig_test_00400_resized_no_blur.jpg
4034      2  Convertible_train_orig_test_00689_resized_no_b...
...     ...                                                ...
3152      3   Pickup_train_orig_test_05681_resized_no_blur.jpg
2297      0    Sedan_train_orig_test_07880_resized_no_blur.jpg
3193      2  Convertible_train_orig_train_08086_resized_no_...
2115      0   Sedan_train_orig_train_00353_resized_no_blur.jpg
2308      1     SUV_train_orig_train_06559_resized_no_blur.jpg

[4802 rows x 2 columns]
      Class                                harmonized_filename
5         1     SUV_train_orig_train_04226_resized_no_blur.jpg
6         0   Sedan_train_orig_train_07233_resized_no_blur.jpg
7         1     SUV_train_orig

In [231]:
if sample_run:
    train_image_paths = ['../../../Images/train/No Blur/' + hf for hf in train_df['harmonized_filename'][:2]]
    train_image_labels = list(train_df['Class'][:2])
    val_image_paths = ['../../../Images/train/No Blur/' + hf for hf in val_df['harmonized_filename'][:2]]
    val_image_labels = list(val_df['Class'][:2])
else:
    train_image_paths = ['../../../Images/train/No Blur/' + hf for hf in train_df['harmonized_filename']]
    train_image_labels = list(train_df['Class'])
    val_image_paths = ['../../../Images/train/No Blur/' + hf for hf in val_df['harmonized_filename']]
    val_image_labels = list(val_df['Class'])

# Check path and label
print(train_image_paths[0])
print(train_image_labels[0])
print(val_image_paths[0])
print(val_image_labels[0])

# Check counts
print(len(train_image_paths))
print(len(train_image_labels))
print(len(val_image_paths))
print(len(val_image_labels))

../../../Images/train/No Blur/SUV_train_orig_test_03377_resized_no_blur.jpg
3
../../../Images/train/No Blur/SUV_train_orig_train_04226_resized_no_blur.jpg
1
4802
4802
1201
1201


## PyTorch Dataset and Preprocessing

In [232]:
# Define the custom dataset
class CustomImageDataset(Dataset):

    # Initialize the dataset object with paths, labels and transform
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    # Get the length of the dataset
    def __len__(self):
        return len(self.image_paths)

    # Get an item from the dataset
    def __getitem__(self, idx):

        # Read the image
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        
        # Apply the transform
        if self.transform:
            image = self.transform(image)
        
        # Get the label
        label = self.labels[idx]

        # Return the image
        return image, label

# Standard ResNet preprocessing
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [233]:
# Create the training dataset
train_dataset = CustomImageDataset(image_paths=train_image_paths, labels=train_image_labels, transform=preprocess)
print(train_dataset[0])

(tensor([[[ 0.8618,  1.1015,  1.0673,  ...,  0.8276,  0.6734,  0.7419],
         [ 0.8618,  1.1015,  1.0673,  ...,  0.8276,  0.6734,  0.7591],
         [ 0.8789,  1.1187,  1.0502,  ...,  0.8276,  0.6906,  0.7762],
         ...,
         [-0.4911, -0.5082, -0.5253,  ..., -0.2856, -0.2856, -0.3027],
         [-0.5082, -0.5253, -0.5424,  ..., -0.2856, -0.2856, -0.3027],
         [-0.5424, -0.5596, -0.5767,  ..., -0.2856, -0.2856, -0.3027]],

        [[ 1.0280,  1.2731,  1.2381,  ...,  0.9930,  0.8354,  0.9055],
         [ 1.0280,  1.2731,  1.2381,  ...,  0.9930,  0.8354,  0.9230],
         [ 1.0455,  1.2906,  1.2206,  ...,  0.9930,  0.8529,  0.9405],
         ...,
         [-0.5301, -0.5476, -0.5651,  ..., -0.3200, -0.3200, -0.3375],
         [-0.5476, -0.5651, -0.5826,  ..., -0.3200, -0.3200, -0.3375],
         [-0.5826, -0.6001, -0.6176,  ..., -0.3200, -0.3200, -0.3375]],

        [[ 1.1062,  1.3502,  1.3154,  ...,  1.0714,  0.9145,  0.9842],
         [ 1.1062,  1.3502,  1.3154,  ...,  

In [234]:
# Create the validation dataset
val_dataset = CustomImageDataset(image_paths=val_image_paths, labels=val_image_labels, transform=preprocess)
print(val_dataset[0])

(tensor([[[-1.1418, -0.4054,  0.7077,  ...,  1.0159,  1.1872,  0.7419],
         [-1.0219, -0.8164,  0.0056,  ...,  1.5297,  1.6838,  1.3242],
         [-0.5424, -0.7479, -0.7308,  ...,  1.5125,  1.6495,  1.4098],
         ...,
         [-0.1657, -0.9534, -0.4397,  ...,  0.1426,  1.0331,  0.7762],
         [-0.3369, -0.5938, -0.0116,  ...,  0.1939,  0.3652,  0.9474],
         [-0.3369, -0.8335, -0.0629,  ...,  0.3994,  1.1529, -1.2445]],

        [[-0.4776,  0.2227,  1.3431,  ...,  1.2381,  1.4132,  0.9580],
         [-0.6176, -0.3901,  0.4503,  ...,  1.7633,  1.9209,  1.5532],
         [-0.4251, -0.6352, -0.5826,  ...,  1.7808,  1.9209,  1.6758],
         ...,
         [-0.1450, -0.9503, -0.4251,  ...,  0.2577,  1.1681,  0.9055],
         [-0.3200, -0.5826,  0.0126,  ...,  0.3102,  0.4853,  1.0805],
         [-0.3200, -0.8277, -0.0399,  ...,  0.5553,  1.3256, -1.1253]],

        [[ 0.0082,  0.7228,  1.8034,  ...,  1.4722,  1.6465,  1.1934],
         [-0.1138,  0.0779,  0.9145,  ...,  

In [235]:
# Print length of train and val datasets
print(len(train_dataset))
print(len(val_dataset))

4802
1201


In [236]:
# Set up the data loaders
batch_size = 32
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

## Load Model

In [237]:
# Load the model
model = models.resnet50(pretrained=True)
# Modify the final layer for our classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(combined_df['Class'].unique()))
# Send the model to the device
model = model.to(device)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [238]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Define the number of epochs
num_epochs = 10

# Train the model
for epoch in range(num_epochs):

    # Start timer
    start_time = time.time()

    # Train the model on the training set
    model.train()
    train_loss = 0.0
    train_correct = 0
    # Iterate over batches
    for inputs, labels in train_data_loader:
        
        # Move the data to the device
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + loss calc + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Update the training loss and accuracy
        # Multiply loss.item() (average loss in the batch) by batch size to get total loss
        train_loss += loss.item() * inputs.size(0)
        # Get the predicted class as the column index with the highest score
        _, preds = torch.max(outputs, 1)
        # Add the number of correct predictions to the total correct
        train_correct += torch.sum(preds == labels.data).item()

    # Evaluate the model on the validation set
    model.eval()
    val_loss = 0.0
    val_correct = 0
    # No gradients
    with torch.no_grad():
        # Iterate over batches
        for inputs, labels in val_data_loader:

            # Move the data to the device
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Update the test loss and accuracy
            # Multiply loss.item() (average loss in the batch) by batch size to get total loss
            val_loss += loss.item() * inputs.size(0)
            # Get the predicted class as the column index with the highest score
            _, preds = torch.max(outputs, 1)
            # Add the number of correct predictions to the total correct
            val_correct += torch.sum(preds == labels.data).item()

    # Print the training and test loss and accuracy
    # Renormalize the loss
    train_loss = train_loss / len(train_dataset)
    val_loss = val_loss / len(val_dataset)
    # Calculate accuracy
    train_acc = train_correct / len(train_dataset)
    val_acc = val_correct / len(val_dataset)
    # End timer
    end_time = time.time()
    # Print
    print(f"Epoch [{epoch + 1}/{num_epochs}] Time Taken: {str(round(end_time - start_time, 2)) + 's'} Train Loss: {train_loss:.4f} Train Acc: {train_acc:.4f} Val Loss: {val_loss:.4f} Val Acc: {val_acc:.4f}")


KeyboardInterrupt: 

## Save Model

In [ ]:
# Save model
torch.save(model.state_dict(), '../../../Output/Classifier Fitting/ResNet/resnet.pth')